## Importing xlsx

In [1]:
%pip install openpyxl
%pip install xlrd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from apifunctions import *

pd.set_option("display.max_rows", None, "display.max_columns", None)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/danigomezlechonbarrachina/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [3]:
wacc=pd.read_excel("../data/datos_Bloomberg/wacc.xlsx")
prices=pd.read_excel("../data/datos_Bloomberg/prices.xlsx")
sales_growth=pd.read_excel("../data/datos_Bloomberg/sales_growth.xlsx")
parameters=pd.read_excel("../data/datos_Bloomberg/parameters.xlsx")

## ID's as index

In [4]:
datasets=[wacc,prices,sales_growth]

for i in datasets:
    i.set_index("ID",inplace=True)

## Dealing with NANs

In [5]:
# We first want to delete the rows which have all NANs for the wacc, prices and sales_growth dataframes

datasets=[wacc,prices,sales_growth]

for i in datasets:
    i.dropna(axis=0, how='all',inplace=True)   

#### For simplicity, in prices we are going to delete all the rows that have nans as that means the company was not trading at that point so it wont allow us to do the backtesting:

In [6]:
prices.dropna(axis=0,inplace=True)

#### Now we are going to delete the companies for which we dont have information in one of the three datasets

In [10]:
remove_columns(prices,wacc,sales_growth)
remove_columns(prices,wacc,sales_growth)

#### To deal with the rest of NANs in wacc and sales_growth we are going to fill them with the average of the values in its time range

In [20]:
fill_na_mean(wacc)
fill_na_mean(sales_growth)

#### At this point we have cleaned the three dataframes: wacc, prices, sales growth. Now we need to deal with the parameters one

### Parameters dataframe